In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import joblib

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import shap
import warnings
import json
import re

warnings.filterwarnings("ignore")
plt.rcParams.update({'font.size': 14})

In [ ]:
class baseline(nn.Module):
    def __init__(self, num):
        super(baseline, self).__init__()
        if(num <= 32):
            self.network = nn.Sequential(
                nn.Linear(num, 16),
                nn.BatchNorm1d(16),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(16, 8),
                nn.BatchNorm1d(8),
                #             nn.Dropout(p=0.1),
                nn.ReLU(),
                nn.Linear(8, 1),
                nn.Sigmoid()
            )
        else:
            self.network = nn.Sequential(
                nn.Linear(num, 32),
                nn.BatchNorm1d(32),
                nn.Dropout(p = 0.4),
                nn.Linear(32, 16),
                nn.BatchNorm1d(16),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(16, 8),
                nn.BatchNorm1d(8),
                #             nn.Dropout(p=0.1),
                nn.ReLU(),
                nn.Linear(8, 1),
                nn.Sigmoid()
            )

    def forward(self, x):
        return self.network(x)

In [ ]:
name = 'Sample data'

path = './' + name

In [ ]:
data = pd.read_csv(path + '/Sample data.csv')
# data.info()
# data['index'].value_counts()
data.head()

In [ ]:
#feature selection
with open(path + '/feature.json', 'r') as f:
    dic = json.load(f)
dx_index = [int(x) for x in dic['dx']]
zl_index = [int(x) for x in dic['zl']]

feature = data.columns[1:]
data[feature] = (data[feature] - data[feature].mean()) / data[feature].std()


dx = data.iloc[:, 38:]
zl = data.iloc[:, 1:38]

dx= dx.iloc[:, dx_index]
zl = zl.iloc[:, zl_index]

ndata = pd.concat([zl, dx, data.iloc[:, 0]], axis=1)

feature_size = len(dx_index) + len(zl_index)
x_train, x_test, y_train, y_test = train_test_split(ndata.iloc[:,:feature_size], ndata.iloc[:,feature_size], test_size = 0.2, random_state=2)     #构造训练集和测试集

x_train_ts = torch.tensor(x_train.values).to(torch.float32)
y_train_ts = torch.tensor(y_train.values).to(torch.float32)
y_train_ts = y_train_ts.reshape(y_train_ts.size()[0], 1)

x_test_ts = torch.tensor(x_test.values).to(torch.float32)
y_test_ts = torch.tensor(y_test.values).to(torch.float32)
y_test_ts = y_test_ts.reshape(y_test_ts.size()[0], 1)

tmp = torch.cat([x_train_ts, y_train_ts], dim = 1)
tmp.size()

## Neural Network

In [ ]:
network_path = path + '/neural_network.pth'

def getScore():
    model.eval()
    rs = model(x_test_ts).detach().numpy().reshape(x_test_ts.size()[0])
    pre = [0 if x < 0.05 else 1 for x in rs]
    acc = accuracy_score(y_test_ts, pre)
    model.train()
    return acc

epochs = 1000
lr = 1e-2
model = baseline(feature_size)
model.train()
criteon = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

loss_ls = []
label = []
num = 1
ans = 0

flag = True
flag1 = True
for epoch in range(epochs):
    target = y_train
    target_hat = model(x_train_ts)
    loss = criteon(target_hat, y_train_ts)

    loss_ls.append(loss.item())
    label.append(num)
    num += 1
#     if(loss.item() <= 0.01):
#         break;

    acc = getScore()
    if(acc > ans):
        ans = acc
        torch.save(model.state_dict(),network_path)
        print('loss{}, acc: {}'.format(loss.item(), ans))
        

    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(epoch, 'loss:', loss.item())

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.figure(figsize = (8,8))
plt.plot(label, loss_ls)
plt.show();

In [ ]:
model.load_state_dict(torch.load(network_path))

model.eval()
rs = model(x_test_ts).detach().numpy().reshape(x_test_ts.size()[0])
pre = [0 if x < 0.05 else 1 for x in rs]
print("acc：{}".format(accuracy_score(y_test_ts, pre)))
print(classification_report(y_test_ts, pre))

rs_train = model(x_train_ts).detach().numpy().reshape(tmp.size()[0])
pre_train = [0 if x < 0.05 else 1 for x in rs_train]
print("acc：{}".format(accuracy_score(y_train_ts, pre_train)))
print(classification_report(y_train, pre_train))

In [ ]:
ROC_path = path + '/ROC.jpg'

fpr, tpr, thresholds = roc_curve(y_test_ts, rs)

plt.figure(figsize = (8,8))
plt.plot(fpr, tpr, label='Neural Network (area = %0.2f)' % metrics.auc(fpr,tpr))
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-specificity')
plt.ylabel('sensitivity')
# plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
# plt.savefig(ROC_path,dpi=300,bbox_inches='tight')
plt.show();

In [ ]:
f = lambda x : model(torch.tensor(x).to(torch.float32)).detach().numpy()

explainer = shap.KernelExplainer(f, x_train.values)

data_for_prediction = x_test.iloc[3,:]
shap_values = explainer.shap_values(data_for_prediction)

shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0], data_for_prediction)

In [ ]:
file_path =  path + '/deep_shap.csv'
# tmp = pd.DataFrame(shap_values[0], columns=x_train.columns, index=x_train.index)
# tmp.to_csv(file_path)
shap_values = pd.read_csv(file_path, index_col=0)
shap_values = shap_values.to_numpy()

In [ ]:
tmp = pd.DataFrame(shap_values, columns=x_train.columns, index=x_train.index)
tmp = tmp.apply(lambda x : abs(x))
tmp = tmp.apply(lambda x : x.mean()).sort_values(ascending = False)
nn_feature = tmp.index

In [ ]:
# shap_values = explainer.shap_values(x_train)
tmp = shap.summary_plot(shap_values, x_train)
# plt.savefig(path + "./deep_shap.jpg",bbox_inches='tight') 
tmp.show();

## Random Forest

In [ ]:
model = RandomForestClassifier(random_state = 0)
# model = LogisticRegression()
model.fit(x_train, y_train)
pre = model.predict(x_test)
print("acc：{}".format(accuracy_score(y_test, pre)))
print(classification_report(y_test, pre))

pre_train = model.predict(x_train)
print("acc：{}".format(accuracy_score(y_train, pre_train)))
print(classification_report(y_train, pre_train))

#ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, pre)

plt.figure(figsize = (8,8))
plt.plot(fpr, tpr, label='Random Forest (area = %0.2f)' % metrics.auc(fpr,tpr))
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-specificity')
plt.ylabel('sensitivity')
# plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig(path + '/ROC.jpg',dpi=300,bbox_inches='tight')
plt.show();

In [ ]:
explainer = shap.KernelExplainer(model.predict_proba, x_train)

data_for_prediction = x_train.iloc[6,:]
shap_values = explainer.shap_values(data_for_prediction)

shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

In [ ]:
file_path = path + '/machine_shap.csv'
# tmp = pd.DataFrame(shap_values[1], columns=x_train.columns, index=x_train.index)
# tmp.to_csv(file_path)
shap_values = pd.read_csv(file_path, index_col=0)
shap_values = shap_values.to_numpy()

In [ ]:
# shap_values = explainer.shap_values(x_train)
tmp = shap.summary_plot(shap_values, x_train)
# plt.savefig(path + '/RF_shap.jpg',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
tmp = pd.DataFrame(shap_values, columns=x_train.columns, index=x_train.index)
tmp = tmp.apply(lambda x : abs(x))
tmp = tmp.apply(lambda x : x.mean()).sort_values(ascending = False)
rf_feature = tmp.index

## Biomarker Discovery

In [ ]:
nn_feature = nn_feature[:20]
rf_feature = rf_feature[:20]

marker = nn_feature.intersection(rf_feature)
print('get {} biomarkers'.format(len(marker)))
print(marker)

col_importance = list(marker)

In [ ]:
sv = pd.DataFrame(shap_values, columns=x_train.columns, index=x_train.index)
sv = sv[col_importance]

def func(tmp):
    f_med = tmp.quantile(0.25)
    med = tmp.quantile(0.5)
    l_med = tmp.quantile(0.75)
    iqr = l_med - f_med
    up = l_med + 1.5 * iqr
    low = f_med - 1.5 * iqr

    sc = tmp.apply(lambda x : -2 if x < low else(
                                            -1 if x < f_med else(
                                            0 if x < l_med else(
                                            1 if x < up else 2))))
    return sc
    

score = sv.apply(func, axis = 0)
score.columns = [x + '_sc' for x in score.columns]
score.head()
#getAllscore
score['allscore'] = score.apply(lambda x : x.sum(), axis = 1)
score['target'] = y_train

score.head()

In [ ]:
coef = []
for col in col_importance:
    s_col = col + '_sc'
    tmp1 = x_train[col]
    tmp2 = score[s_col]
    coef.append(np.corrcoef(tmp1, tmp2)[0][1])

dic = {'col' : col_importance, 'coef' : coef}
coef = pd.DataFrame(dic)
coef

In [ ]:
bio_marker = list(coef[(coef['coef'] >= 0.7) | (coef['coef'] <= -0.7)]['col'])
bio_marker_sc = [x + '_sc' for x in bio_marker]
score1 = score[bio_marker_sc]
score1['allscore'] = score.apply(lambda x : x.sum(), axis = 1)
score1['target'] = y_train
score1.head()

In [ ]:
X = score1['allscore']
y = score1['target']
fpr, tpr, thresholds = roc_curve(y, X)

plt.figure(figsize = (8,8))
plt.plot(fpr, tpr, label='AUROC = %0.2f' % metrics.auc(fpr,tpr))
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-specificity')
plt.ylabel('sensitivity')
# plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('ROC.jpg',dpi=300,bbox_inches='tight')
plt.show();

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
# print('best cutoff:', optimal_threshold)

y_pred = (X >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

OR = (tp * tn) / (fp * fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

In [ ]:
data = pd.read_csv(path + '/Sample data.csv')
feature = data.columns[0:]

x_train, x_test, y_train, y_test = train_test_split(data[bio_marker], data.iloc[:,0], test_size = 0.2, random_state=2)     #构造训练集和测试集

# x_train = data[col_importance]
# x_train = x_test
d1 = pd.concat([x_train, score1], axis = 1)

In [ ]:
qjsc = pd.DataFrame(columns=['name','score','min','max'])
pre = -1

pre_name = -1
def func(tmp):
#     print(tmp.iloc[0,1])
    global pre, pre_name
    name = tmp.columns[0]
    if(pre_name != name):
        pre_name = name
        low = 0
    else:
        low = pre
    tmp = tmp.iloc[:, 0]
    f_med = tmp.quantile(0.25)
    med = tmp.quantile(0.5)
    l_med = tmp.quantile(0.8)
    iqr = l_med - f_med
    up = l_med + 1.5 * iqr
    
    pre = l_med
    
    
    t = pd.DataFrame(columns=['min','max'])
        
    t['min'] = [low]
    t['max'] = [l_med]
        
    return t

for name in bio_marker:
    sc_name = name + '_sc'
    if(coef[coef['col'] == name]['coef'].values[0] > 0):
        tmp = d1[[name, sc_name]].groupby(sc_name)
        rt = tmp.apply(func)
    else:
        tmp = d1[[name, sc_name]].sort_values(sc_name, ascending = False)
        tmp = tmp.groupby(sc_name, sort = False)
        rt = tmp.apply(func)
    rt.index = tmp.min().index
    rt.reset_index(inplace = True)
    rt['name'] = sc_name
    rt.rename({sc_name:'score'}, inplace = True, axis = 1)
    rt = rt[['name','score','min','max']]
    qjsc = qjsc.append(rt)
#     print(d1[[name, sc_name]].groupby(sc_name)[name].apply(lambda x : x.describe(percentiles = [.2,.25,.5,.75,.8])))


# qjsc

In [ ]:
def f1(src, sc):
#     print(sc)
#     print(src)
    for index in sc['score'].unique():
        tmp = sc[sc['score'] == index]
#         print(tmp.iloc[0, 2])
        if(src >= tmp.iloc[0, 2] and src < tmp.iloc[0, 3]):
            return index
    
    return 0

def func(tmp):
    name = tmp.name + '_sc'
    sc = qjsc[qjsc['name'] == name]
    sc_tmp = tmp.apply(f1, sc = sc)
    return sc_tmp
    
tmp = x_train.apply(func)

# tmp.replace(-1,0, inplace = True)
# tmp.replace(-2, 0, inplace = True)
tmp['allscore'] = tmp.apply(lambda x: x.sum(), axis = 1)

In [ ]:
X = tmp['allscore']
y = y_train
# y = y_test
# y = data.iloc[:, 0]
fpr, tpr, thresholds = roc_curve(y, X)

plt.figure(figsize = (8,8))
plt.plot(fpr, tpr, label='AUROC = %0.2f' % metrics.auc(fpr,tpr))
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-specificity')
plt.ylabel('sensitivity')
# plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('ROC.jpg',dpi=300,bbox_inches='tight')
plt.show();

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
# print('best cutoff:', optimal_threshold)

y_pred = (X >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

OR = (tp * tn) / (fp * fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)